<a href="https://colab.research.google.com/github/rurusasu/Python/blob/master/Colab_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive をマウントする．
REF: [Google ドライブにマウントし、ファイルへアクセスする方法](https://blog.kikagaku.co.jp/google-colab-drive-mount)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Colab と VS Code を使って python の開発環境を構築する際に使用するスクリプト

REF: [Google ColabとVSCodeを用いた分析環境運用方法 〜kaggle Tipsを添えて〜](https://www.takapy.work/entry/2021/08/17/185047)

In [6]:
!cat /etc/lsb-release
!nvcc -V
!nvidia-smi

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION="Ubuntu 18.04.5 LTS"
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Sat Mar  5 01:40:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8   

# ngrok のインストール・設定

In [7]:
!apt-get -y update

!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip

# sshの設定
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Hit:13 http://ppa.launchpad.net/cran/

# ssh サーバーの起動

In [8]:
import random, string

# Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

# Set root password
!echo root:$password | chpasswd
!mkdir -p /var/run/sshd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
!echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

# アクセストークンの設定
# https://dashboard.ngrok.com/auth/your-authtoken
# authtoken="取得したYour Authtokenの値"
authtoken = "25sK8o5a2l937Xqh6wSfYZaUOTY_3S7rba6ghRjFi3tndyW6k"

# Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

# Colab サーバーの環境設定

In [9]:
# シンボリックリンクを作成
!ln -sfn /content/drive/MyDrive/workspace /root/workspace

# zshのインストール
!sudo apt-get install zsh

# oh-my-zshのインストール
!wget https://github.com/robbyrussell/oh-my-zsh/raw/master/tools/install.sh -O - | zsh || true

# zshをデフォルトに設定
!chsh -s /usr/bin/zsh

# bashファイルの作成
!echo 'export PATH=/usr/local/cuda/bin:$PATH' >> /root/.bash_profile
!echo 'export LD_LIBRARY_PATH=/usr/lib64-nvidia' >> /root/.bash_profile
!echo 'export PROMPT_COMMAND="history -a"' >> /root/.bash_profile
!echo 'export HISTFILE=/root/.zsh-history' >> /root/.bash_profile
!echo 'export PYTHONDONTWRITEBYTECODE=1' >> /root/.bash_profile
!echo 'export TF_CPP_MIN_LOG_LEVEL=2' >> /root/.bash_profile

# ファイルをgoogle driveからサーバーへコピー
!cp /content/drive/MyDrive/workspace/.gitconfig .gitconfig
!cp /content/drive/MyDrive/workspace/.zshrc .zshrc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zsh is already the newest version (5.4.2-3ubuntu3.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 75 not upgraded.
--2022-03-05 01:41:26--  https://github.com/robbyrussell/oh-my-zsh/raw/master/tools/install.sh
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ohmyzsh/ohmyzsh/raw/master/tools/install.sh [following]
--2022-03-05 01:41:26--  https://github.com/ohmyzsh/ohmyzsh/raw/master/tools/install.sh
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh [following]
--2

# 接続情報の取得

In [10]:
import urllib.request, json, getpass
# Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')

print(f"""Host google-colab-ssh
HostName {host}
Port {port}
User root
""")

# Print root password
print(f'Root password: {password}')

Host google-colab-ssh
HostName 8.tcp.ngrok.io
Port 11388
User root

Root password: 7M99ej44KzefS6UJ2cVi
